In [ ]:
#@title RAGデータ確認用_検索順位評価ツール

# ------------------------------------------------
# 1. セットアップとライブラリのインストール
# ------------------------------------------------
!pip install -q openai numpy scipy ipywidgets

import os
import csv
import json
import numpy as np
import openai
from scipy.spatial.distance import cosine
import ipywidgets as widgets
from google.colab import files
from google.colab import userdata
from IPython.display import display, clear_output
import re

# ------------------------------------------------
# 2. APIキーのセットアップとクライアントの初期化
# ------------------------------------------------
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        raise ValueError("APIキーが設定されていません。Colabのシークレット機能で'OPENAI_API_KEY'を設定してください。")
    openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)
    print("✅ OpenAI APIキーの読み込みとクライアントの初期化が完了しました。")
except Exception as e:
    print(f"❌ エラー: {e}")

# ------------------------------------------------
# 3. グローバル変数 (アプリケーションの状態管理)
# ------------------------------------------------
documents = [] # 形式: [{"id": int, "text": str, "metadata": dict}, ...]
doc_embeddings = None
last_embedding_model = None
last_division_settings = None
file_name = "uploaded_data"
current_query = ""

# ------------------------------------------------
# 4. UIコンポーネントの定義
# ------------------------------------------------
file_uploader = widgets.FileUpload(accept='.txt,.json', multiple=False, description='ファイルをアップロード')
division_method_selector = widgets.Dropdown(
    options=[('1行を1ドキュメントとして分割', 'line'), ('固定長のチャンクに分割', 'chunk'), ('JSONオブジェクトごとに分割', 'json_object')],
    value='line', description='分割方法:', style={'description_width': 'initial'}
)
json_content_key_input = widgets.Text(value='content', placeholder='コンテンツのキー名', description='JSONコンテンツキー:', layout=widgets.Layout(display='none'))
chunk_size_input = widgets.IntText(value=500, description='チャンクサイズ:', layout=widgets.Layout(display='none', width='200px'))
chunk_overlap_input = widgets.IntText(value=50, description='オーバーラップ:', layout=widgets.Layout(display='none', width='200px'))
division_settings_box = widgets.HBox([chunk_size_input, chunk_overlap_input, json_content_key_input])
embedding_model_selector = widgets.Dropdown(
    options=[('使用しない (キーワード検索)', 'none'), ('text-embedding-3-small', 'text-embedding-3-small'), ('text-embedding-3-large', 'text-embedding-3-large'), ('text-embedding-ada-002', 'text-embedding-ada-002')],
    value='text-embedding-3-small', description='埋め込みモデル:', style={'description_width': 'initial'}
)
query_input = widgets.Text(value='', placeholder='検索クエリを入力...', description='クエリ:', layout=widgets.Layout(width='80%'))
search_button = widgets.Button(description='検索実行', button_style='success', icon='search')
output_area = widgets.Output()
download_area = widgets.Output()

# ------------------------------------------------
# 5. ヘルパー関数 (コアロジック)
# ------------------------------------------------
def get_embedding(text, model="text-embedding-3-small"):
    if not text or not isinstance(text, str): return None
    try:
        return openai_client.embeddings.create(input=[text.replace("\n", " ")], model=model).data[0].embedding
    except Exception as e:
        with output_area: print(f"埋め込み取得エラー: {e}")
        return None

def split_text_into_chunks(text, chunk_size, chunk_overlap):
    if chunk_size <= chunk_overlap: raise ValueError("チャンクサイズはオーバーラップより大きくする必要があります。")
    chunks, start_index = [], 0
    while start_index < len(text):
        end_index = start_index + chunk_size
        chunks.append(text[start_index:end_index])
        start_index += chunk_size - chunk_overlap
    return chunks

def update_documents(new_docs):
    global documents, doc_embeddings, last_division_settings
    documents = new_docs
    doc_embeddings, last_division_settings = None, None
    with output_area: print(f"✅ {len(documents)} 件のドキュメントを生成しました。")
    display_source_document_downloader()

def display_source_document_downloader():
    with download_area:
        clear_output()
        if documents:
            button = widgets.Button(description=f"分割された全{len(documents)}件のドキュメントをダウンロード", button_style='info', icon='download')
            def download_source(b):
                base_name = os.path.splitext(file_name)[0]
                dl_filename = f"divided_{base_name}.txt"
                content = "\n".join([f"--- 分割ドキュメント ID: {d['id']} ---\n{d.get('text', '')}\n" for d in documents])
                with open(dl_filename, "w", encoding="utf-8") as f: f.write(content)
                files.download(dl_filename)
            button.on_click(download_source)
            display(widgets.VBox([widgets.HTML("<hr>"), button]))

def display_results_downloader(results, query):
    """★復元★: 検索結果をCSVでダウンロードするボタンを表示"""
    with download_area:
        clear_output(wait=True)
        display_source_document_downloader()
        if results:
            button = widgets.Button(description=f"全{len(results)}件の検索結果をCSVでダウンロード", button_style='success', icon='download')
            def download_csv(b):
                query_sanitized = re.sub(r'[\\/*?:"<>|]', "", query)[:20]
                dl_filename = f"search_results_{query_sanitized}.csv"
                with open(dl_filename, 'w', newline='', encoding='utf-8-sig') as f:
                    writer = csv.writer(f)
                    writer.writerow(['Rank', 'Split_ID', 'Score', 'Text'])
                    for i, r in enumerate(results):
                        score_str = f"{r['score']:.6f}" if isinstance(r['score'], float) else str(r['score'])
                        writer.writerow([i + 1, r['doc_info']['id'], score_str, r['doc_info']['text']])
                files.download(dl_filename)
            button.on_click(download_csv)
            display(button)

# ------------------------------------------------
# 6. イベントハンドラ (UIの動作を定義)
# ------------------------------------------------
def on_division_method_change(change):
    method = change['new']
    chunk_size_input.layout.display = 'flex' if method == 'chunk' else 'none'
    chunk_overlap_input.layout.display = 'flex' if method == 'chunk' else 'none'
    json_content_key_input.layout.display = 'flex' if method == 'json_object' else 'none'

def on_file_upload(change):
    # (この関数は正常に動作します)
    global file_name
    uploaded_file = change['new']
    if not uploaded_file: return
    file_info = next(iter(uploaded_file.values()))
    file_name, content_bytes = file_info['metadata']['name'], file_info['content']
    with output_area:
        clear_output()
        print(f"'{file_name}' を読み込み中...")
        try:
            new_docs = []
            method = division_method_selector.value
            if method == 'json_object':
                if not file_name.endswith('.json'): raise ValueError("この分割方法はJSONファイルでのみ使用できます。")
                json_data = json.loads(content_bytes.decode('utf-8'))
                if not isinstance(json_data, list): raise ValueError("JSONデータはオブジェクトのリスト形式である必要があります。")
                if json_data and isinstance(json_data[0], dict):
                    common_keys = ['content', 'text', 'body', 'document', 'description']
                    found_key = next((key for key in common_keys if key in json_data[0]), None)
                    if found_key:
                        json_content_key_input.value = found_key
                        print(f"ℹ️ コンテンツキーとして '{found_key}' を自動検出しました。")
                content_key = json_content_key_input.value
                for i, item in enumerate(json_data):
                    if not isinstance(item, dict): continue
                    metadata = item.get('metadata', {})
                    content = item.get(content_key, "")
                    meta_str = ", ".join([f"{k}: {v}" for k, v in metadata.items()])
                    combined_text = f"メタデータ: [ {meta_str} ]\n内容: {content}"
                    new_docs.append({"id": i, "text": combined_text, "metadata": metadata})
            else:
                full_text = content_bytes.decode('utf-8')
                if method == 'chunk':
                    chunks = split_text_into_chunks(full_text, chunk_size_input.value, chunk_overlap_input.value)
                    new_docs = [{"id": i, "text": chunk, "metadata": {}} for i, chunk in enumerate(chunks)]
                else:
                    lines = full_text.splitlines()
                    new_docs = [{"id": i, "text": line.strip(), "metadata": {}} for i, line in enumerate(lines) if line.strip()]
            update_documents(new_docs)
        except Exception as e:
            with output_area: print(f"❌ ファイル処理エラー: {e}")
            with download_area: clear_output()

def on_search_button_clicked(b):
    """★復元★: 検索ボタンがクリックされたときのメイン処理"""
    global documents, doc_embeddings, last_embedding_model, last_division_settings, current_query

    with output_area:
        clear_output()
        current_query = query_input.value
        embedding_model = embedding_model_selector.value
        current_division_settings = (division_method_selector.value, chunk_size_input.value, chunk_overlap_input.value, json_content_key_input.value)

        if not current_query: print("❌ クエリを入力してください。"); return
        if not documents: print("❌ 検索対象のドキュメントがありません。ファイルをアップロードしてください。"); return

        print(f"🔍 検索を開始します...\nクエリ: {current_query}\n埋め込みモデル: {embedding_model}\n" + "-" * 30)

        results = []
        doc_texts = [d['text'] for d in documents]

        if embedding_model == 'none':
            print("キーワード検索を実行中...")
            query_words = current_query.lower().split()
            for doc_info in documents:
                score = sum(1 for word in query_words if word in doc_info['text'].lower())
                results.append({'doc_info': doc_info, 'score': score})
            results.sort(key=lambda x: x['score'], reverse=True)
        else:
            if doc_embeddings is None or last_embedding_model != embedding_model or last_division_settings != current_division_settings:
                print(f"ドキュメントの埋め込みを生成中 (モデル: {embedding_model})...")
                temp_embeddings = [get_embedding(text, embedding_model) for text in doc_texts]

                valid_docs_with_embeddings = [(documents[i], emb) for i, emb in enumerate(temp_embeddings) if emb is not None]
                if not valid_docs_with_embeddings: print("❌ すべてのドキュメントの埋め込み生成に失敗しました。"); return

                valid_docs, doc_embeddings_list = zip(*valid_docs_with_embeddings)
                documents = list(valid_docs)
                doc_embeddings = np.array(doc_embeddings_list)
                last_embedding_model = embedding_model
                last_division_settings = current_division_settings
                print("✅ 埋め込み完了。")

            print("クエリの埋め込みを生成中...")
            query_embedding = get_embedding(current_query, embedding_model)
            if query_embedding is None: print("❌ クエリの埋め込みに失敗しました。"); return

            print("類似度を計算中...")
            similarities = [1 - cosine(query_embedding, doc_emb) for doc_emb in doc_embeddings]
            sorted_indices = np.argsort(similarities)[::-1]
            results = [{'doc_info': documents[i], 'score': similarities[i]} for i in sorted_indices]

        print(f"\n🏆 検索結果 (全{len(results)}件) 🏆\n")
        if not results:
            print("一致する結果は見つかりませんでした。")
        else:
            for i, result in enumerate(results):
                score_str = f"{result['score']:.4f}" if isinstance(result['score'], float) else str(result['score'])
                print(f"【Rank {i+1}】(分割ID: {result['doc_info']['id']}) Score: {score_str}")
                print(f"   📄 {result['doc_info']['text']}")
                print("-" * 20)

    display_results_downloader(results, current_query)

# ------------------------------------------------
# 7. UIの最終的な組み立てとイベントリスナーの登録
# ------------------------------------------------
division_method_selector.observe(on_division_method_change, names='value')
file_uploader.observe(on_file_upload, names='value')
search_button.on_click(on_search_button_clicked)
on_division_method_change({'new': division_method_selector.value})

ui = widgets.VBox([
    widgets.HTML("<h2>検索順位 評価ツール</h2>"),
    widgets.HTML("""
    <p><b>使い方:</b></p>
    <ol>
      <li><b>分割方法</b>を選択します。（JSONの場合は「JSONオブジェクトごと」を選ぶと便利です）</li>
      <li><b>ファイルをアップロード</b>します。JSONの場合、コンテンツのキーが自動で推測されます。</li>
      <li>（任意）分割された全ドキュメントをダウンロードし、意図通りか確認します。</li>
      <li><b>クエリを入力</b>して「検索実行」すると、全件のランキングが表示され、結果をCSVでダウンロードできます。</li>
    </ol>
    """),
    widgets.HBox([file_uploader, division_method_selector]),
    division_settings_box,
    embedding_model_selector,
    widgets.HBox([query_input, search_button]),
    widgets.HTML("<hr>"),
    output_area,
    download_area
])

display(ui)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.4 MB/s eta 0:00:00
✅ OpenAI APIキーの読み込みとクライアントの初期化が完了しました。


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>